<a href="https://colab.research.google.com/github/jvataidee/DataAnalysis/blob/master/Analise_Financeira_Mercado_Gado_de_Corte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# Análise Mercado Financeira de Gado de Corte


---


## Importanto bibliotecas básicas

In [1]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Obtenção dos dados

Encontrar dados para começar as análises sempre é um processo complicado, em especial quando a gente está falando de um eixo de mercado ainda muito artesanal, o qual etá modificando agora seu modo com a Pecuária 4.0, assim, não foi possível encontrar uma API nacional que automatize a obtenção dados sobre o preço das commodities.

Então, para isso realizei a instalação de duas bibliotecas investpy, API criada a parti da plataforma [investing.com](https://investing.com/) que possui diversos bases de dados financeiras, não somente de commodities. Outra biblioteca que foi necessária foi a [Yahoo! Finance](https://finance.yahoo.com/) a qual utilizei para obter os dados referentes a cotas do dollar.


In [ ]:
!pip install investpy -q

In [ ]:
!pip install yfinance -q

In [4]:
pip install -U sidrapy -q

In [5]:
#importar bibliotecas financeiras
import investpy
import yfinance as yf
import sidrapy

### Dados de Commodities

In [6]:
#Lista de commodities
commodities = ["Live Cattle", "Feeder Cattle", ]

#Data de inicio e fim de análise
inicio = "01/07/2014"
fim = "01/07/2021"

In [85]:
#Função para importar dados das commodities
def commodities_df(commodities, inicio,fim):
  precos = pd.DataFrame()
  for i in commodities:
    df = investpy.commodities.get_commodity_historical_data(commodity=i, from_date=inicio, to_date=fim).Close
    df.rename(i, inplace=True)
    df.columns = [i]
    precos = pd.concat([precos,df], axis=1)
    precos.index.name="Date"
  return precos


#Data Frame de commodites
df = commodities_df(commodities,inicio,fim)

#Importar dollar
doll = yf.Ticker("USDBRL=x")
dollar = pd.DataFrame(doll.history(start ="2014-01-01", end ="2020-12-31" ).Close)
dollar.columns = ["Dollar"]
dollar.index.name="Date"

#Concatenar dado
dados =pd.merge(df, dollar, how='inner', right_index=True, left_index=True)
dados.head()

,Live Cattle,Feeder Cattle,Dollar
Date,,,
2014-07-01,151.500,215.78,2.2117
2014-07-02,152.325,216.20,2.1981
2014-07-03,155.000,217.62,2.2223
2014-07-07,154.950,218.00,2.2098
2014-07-08,153.525,215.90,2.2225


In [86]:
dados_trimestre = dados.groupby(pd.Grouper(freq='Q')).mean()
dados_trimestre = dados_trimestre.reset_index()
dados_trimestre.head()

,Date,Live Cattle,Feeder Cattle,Dollar
0,2014-09-30,154.548047,220.610000,2.268325
1,2014-12-31,165.997308,233.692187,2.538731
2,2015-03-31,157.177049,210.525246,2.864362
3,2015-06-30,154.694531,218.881905,3.068503
4,2015-09-30,143.911338,207.060313,3.520658


### Dados do SIDRA

In [103]:
import requests

#link do banco de dados sidra
consulta = 'https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202'
ufs = [5]

#Baixar dados do sidra
df2 = pd.DataFrame()
for uf in ufs:
    url = consulta.format(uf)
    dados_json = requests.get(url).json()
    dados_pandas = pd.DataFrame(dados_json)
    df2 = pd.concat([dados_pandas,df2])
  
#Filtrar dados
sidra = df2.copy() 
colunas = list(df2.iloc[0,:])
sidra.columns = colunas
sidra = sidra.iloc[1:,:]
sidra = sidra[sidra["Variável"]=="Animais abatidos"]
sidra = sidra[sidra["Tipo de inspeção"]=="Federal"]
sidra = sidra.reset_index()
sidra = sidra.iloc[:, [5, 11, 15]]
sidra = sidra.drop(sidra[sidra["Valor"] == 'X'].index)
sidra["Valor"] = sidra["Valor"].astype(float)


,Valor,Trimestre,Tipo de rebanho bovino
0,1376884.0,1º trimestre 1997,Bois
1,867851.0,1º trimestre 1997,Vacas
2,2726534.0,1º trimestre 1997,Total
3,376902.0,1º trimestre 1997,Novilhos
4,101791.0,1º trimestre 1997,Novilhas


In [131]:
#Encontrar datas
times = sidra.Trimestre.str.split(expand=True)
times[2] = pd.to_datetime(times[2])
sidra3 = sidra[times[2] >= "2014-01-01"]
sidra3 = sidra3[times[2] < "2021-01-01"]
sidra3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,Valor,Trimestre,Tipo de rebanho bovino
1360,3312448.0,1º trimestre 2014,Bois
1361,2098007.0,1º trimestre 2014,Vacas
1362,6382106.0,1º trimestre 2014,Total
1363,310997.0,1º trimestre 2014,Novilhos
1364,660654.0,1º trimestre 2014,Novilhas


In [133]:
sidra3[sidra3["Tipo de rebanho bovino"] == "Bois"]

,Valor,Trimestre,Tipo de rebanho bovino
1360,3312448.0,1º trimestre 2014,Bois
1365,1309144.0,1º trimestre 2014,Bois
1370,1006540.0,1º trimestre 2014,Bois
1375,996764.0,1º trimestre 2014,Bois
1380,3639688.0,2º trimestre 2014,Bois
...,...,...,...
1895,1343151.0,3º trimestre 2020,Bois
1900,3903690.0,4º trimestre 2020,Bois
1905,1425225.0,4º trimestre 2020,Bois
1910,1199678.0,4º trimestre 2020,Bois


In [130]:
bois = sidra3[sidra3["Tipo de rebanho bovino"] == "Bois"]
vacas = sidra3[sidra3["Tipo de rebanho bovino"] == "Vacas"]
novilhas = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhos"]
novilhos = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhas"]
Total = sidra3[sidra3["Tipo de rebanho bovino"] == "Totais"]

##Análises exploratória

###Pré-processamento


### Análises da série temporal

## Modelo de previsão

### Tratamento dos dados

##Conclusão